In [1]:
# Import libraries
import numpy as np
import cv2

In [2]:
# load the data
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask.shape

(1000, 50, 50, 3)

In [4]:
without_mask.shape

(1000, 50, 50, 3)

In [5]:
# Reshape the data
with_mask = with_mask.reshape(1000, 50*50*3)
without_mask = without_mask.reshape(1000, 50*50*3)

In [6]:
with_mask.shape

(1000, 7500)

In [7]:
without_mask.shape

(1000, 7500)

In [8]:
# Concatenate the numpy arrays
X = np.r_[without_mask, with_mask]

In [9]:
X.shape

(2000, 7500)

In [10]:
labels = np.zeros(X.shape[0])  # label:0 for without mask 

In [11]:
labels[1000:] = 1.0  # label:1 for mask 

In [12]:
names = {0.0 : "No Mask", 1.0: "Mask"}

In [13]:
y = labels

## Model Training

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# Train Test split with 80% training data and 20% test data
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=101)

In [17]:
X_train.shape

(1600, 7500)

In [18]:
X_test.shape

(400, 7500)

### Dimensionality Reduction
We have 7500 columns which will slow down the training speed during Machine Learning, so we need to reduce the dimensionality

In [19]:
from sklearn.decomposition import PCA

In [20]:
pca = PCA(n_components=4)
X_train = pca.fit_transform(X_train)

In [21]:
X_train[0]

array([-1736.85871353,  1318.19906773,   228.70070339,   770.96868592])

In [22]:
 X_train.shape

(1600, 4)

In [24]:
# Train the model 
svm = SVC(C=0.01)
svm.fit(X_train, y_train)

SVC(C=0.01)

In [25]:
# Prediction
X_test = pca.transform(X_test)
y_pred = svm.predict(X_test)

In [26]:
accuracy_score(y_test, y_pred)

1.0

# Implementation on Live Camera data

In [31]:
names = {0.0: "No Mask", 1.0: "Mask"}

haar_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(1) # 1 (for external camera)
font = cv2.FONT_HERSHEY_COMPLEX
data = []

# Define the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Specify the codec
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))  # Adjust resolution and frame rate as needed

while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 4)
            face = img[y:y + h, x:x + w, :]
            face = cv2.resize(face, (50, 50))
            face = face.reshape(1, -1)
            face = pca.transform(face)
            pred = svm.predict(face)
            n = names[int(pred)]
            
            # Set different colors based on the value of n
            color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
            
            cv2.putText(img, n, (x, y), font, 1, color, 2)
        
        # Save the frame to the video file
        out.write(img)
        
        cv2.imshow('result', img)
        
        # 27 = ASCII code of Escape
        if cv2.waitKey(2) == 27:
            break

capture.release()
out.release()  # Release the video writer
cv2.destroyAllWindows()

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

...

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  color = (0, 255, 0) if int(pred) == 1 else (0, 0, 255)
C:\Users\Asus\AppData\Local\Temp\ipykernel_8304\2758640027.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = names[int(pred)]
C:\Users\Asus\AppData\Local\Temp\ipy

In [32]:
!pip list

Package                      Version
---------------------------- --------
absl-py                      1.4.0
anyio                        3.7.1
appdirs                      1.4.4
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
binaryornot                  0.4.4
bleach                       6.0.0
blinker                      1.6.2
cachetools                   5.3.1
certifi                      2023.5.7
cffi                         1.15.1
chardet                      5.2.0
charset-normalizer           3.2.0
click                        8.1.7
colorama                     0.4.6
comm                         0.1.3
contourpy                    1.1.0
cookiecutter                 2.3.0
cycler                       0.11.0
debugpy                      1.6.7
decora